In [20]:
from __future__ import absolute_import
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import argparse
import csv
import io
import logging
import pandas as pd

def get_csv_reader(readable_file = 'Data/reviews-kopie-2.csv'):
    field_list = ['date', 'comments']
    # Open a channel to read the file from GCS
    #gcs_file = beam.io.filesystems.FileSystems.open(readable_file)
    dataframe = pd.read_csv(readable_file, sep=",", error_bad_lines=False)
    # Return the csv reader
    return dataframe #csv.DictReader(io.TextIOWrapper(gcs_file), field_list)
data = get_csv_reader()

b'Skipping line 9: expected 6 fields, saw 7\n'


In [21]:
data.dtypes

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

In [22]:
import math
def calculate_sentiment(entry):
    sid_obj = SentimentIntensityAnalyzer() 
    if (type(entry) != str and math.isnan(entry)):
        return -55
    opinion = sid_obj.polarity_scores(entry)
    return opinion['compound']


data['comments_'] = data['comments'].apply(calculate_sentiment)
data = data[data['comments_'] != -55]
data = data.groupby('date')['comments'].mean()

DataError: No numeric types to aggregate

In [ ]:
data

In [ ]:
class MyPredictDoFn(beam.DoFn):

    def __init__(self):
        self._model = SentimentIntensityAnalyzer()
        self.entry = None
        
    def calculate_sentiment(self, entry):
        sid_obj = SentimentIntensityAnalyzer() 
        if (type(self.entry) != str and math.isnan(self.entry)):
            return -55
            opinion = sid_obj.polarity_scores(self.entry)
        return opinion['compound']

    def process(self, elements, **kwargs):

        df = pd.DataFrame(elements)

        df['comments'] = df['comments'].apply(calculate_sentiment)
        df = df[df['comments'] != -55]
        df = df.groupby('date')['comments'].mean()
        
        return df